# Hub Joint Comparison for Cuboid Struts

This notebook compares different hub joint strategies for geodesic domes with cuboid (rectangular cross-section) struts.

## Hub Styles Compared

1. **Convex Hull** (baseline) - Wraps all strut end corners in a convex hull
2. **Tapered Triangular Prism** - Flat triangular inner face in tangent plane, tapered to outer triangle
3. **Cylindrical Core** - Simple cylinder with miter-cut struts meeting it tangentially

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0, '../src')

import math
import cadquery as cq
from ocp_vscode import show

from chapel2.dome_generator import (
    generate_dome_with_hubs,
    generate_chapel_dome,
)

from chapel2.geometry import (
    generate_honeycomb_dome,
    normalize, sub, add, scale, cross, norm, build_vertex_to_edges_map
)

from chapel2.hubs import (
    compute_hub_geometry,
    create_hub_by_style,
)

from chapel2.cuboid_struts import (
    create_shortened_cuboid_strut
)

from chapel2.miter_struts import (
    create_miter_cut_strut,
    compute_miter_cut_angles,
    analyze_miter_cuts_for_dome,
)


## Section 1: Simple 3-Strut Test Case

First, let's create a simple test case with just one vertex and 3 struts meeting at it.
This makes it easy to inspect the hub geometry in detail.


In [4]:
# Generate a small dome to get a single vertex with 3 struts
radius_cm = 50.0
frequency = 2
strut_width = 3.0  # cm
strut_depth = 3.0  # cm
hub_inset = strut_width * 0.5

vertices, faces, edges = generate_honeycomb_dome(radius_cm, frequency, 0.5)
vertex_to_edges = build_vertex_to_edges_map(edges)

print(f"Dome geometry: {len(vertices)} vertices, {len(edges)} edges")

# Find a vertex with exactly 3 struts (interior vertex)
test_vertex_idx = None
for v_idx, edge_list in vertex_to_edges.items():
    if len(edge_list) == 3:
        test_vertex_idx = v_idx
        break

print(f"Using vertex {test_vertex_idx} with 3 struts")
test_vertex = vertices[test_vertex_idx]


Dome geometry: 58 vertices, 82 edges
Using vertex 55 with 3 struts


In [5]:
# Create the 3 struts meeting at this vertex
dome_center = (0, 0, 0)

connected_edges = vertex_to_edges[test_vertex_idx]
struts_at_vertex = []

for edge_idx in connected_edges:
    v1_idx, v2_idx = edges[edge_idx]
    start = vertices[v1_idx]
    end = vertices[v2_idx]
    
    strut = create_shortened_cuboid_strut(
        start, end, strut_width, strut_depth,
        hub_inset, hub_inset, dome_center
    )
    struts_at_vertex.append(strut)

print(f"Created {len(struts_at_vertex)} struts")

# Show the struts without hub - notice the gap at the vertex
struts_compound = cq.Compound.makeCompound(struts_at_vertex)
print("\nStruts meeting at vertex (without hub) - notice the gap:")
show(struts_compound)


Created 3 struts

Struts meeting at vertex (without hub) - notice the gap:
Using port 3939
+


In [6]:
# Compute hub geometry for this vertex
hub_info = compute_hub_geometry(
    test_vertex, test_vertex_idx, vertices, edges, vertex_to_edges,
    strut_width, strut_depth, hub_inset, dome_center
)

print(f"Hub info:")
print(f"  Vertex position: ({test_vertex[0]:.2f}, {test_vertex[1]:.2f}, {test_vertex[2]:.2f})")
print(f"  Number of struts: {hub_info['num_struts']}")
print(f"  Hub inset: {hub_info['hub_inset']:.2f} cm")


Hub info:
  Vertex position: (44.33, 8.33, -13.91)
  Number of struts: 3
  Hub inset: 1.50 cm


## Section 2: Tapered Prism Hub (Option 2)

The tapered prism hub has:
- **Inner face**: Flat triangle lying in the tangent plane (perpendicular to radial)
- **Outer face**: Another triangle, offset along the radial direction
- **Side faces**: Connect corresponding edges, angled to meet strut end faces


In [7]:
# Create tapered prism hub
tapered_prism_hub = create_hub_by_style(
    hub_info, strut_width, strut_depth, dome_center, 
    hub_style="tapered_prism"
)

if tapered_prism_hub is not None:
    print("Tapered Prism Hub created successfully!")
    print("\nHub only (tapered prism):")
    show(tapered_prism_hub)
else:
    print("Failed to create tapered prism hub")


Tapered Prism Hub created successfully!

Hub only (tapered prism):
+


In [8]:
# Show struts + tapered prism hub together
if tapered_prism_hub is not None:
    print("Struts + Tapered Prism Hub:")
    show(struts_compound, tapered_prism_hub)


Struts + Tapered Prism Hub:
cc


## Section 3: Cylindrical Core Hub (Option 3)

The cylindrical core hub works with miter-cut struts:
- **Hub**: Simple cylinder aligned with radial direction
- **Struts**: Ends cut at compound miter angle so faces lie in radial planes
- **Fit**: Strut end faces meet cylinder tangentially


In [9]:
# Create cylindrical core hub
cylindrical_hub = create_hub_by_style(
    hub_info, strut_width, strut_depth, dome_center,
    hub_style="cylindrical_core"
)

if cylindrical_hub is not None:
    print("Cylindrical Core Hub created successfully!")
    print("\nHub only (cylindrical core):")
    show(cylindrical_hub)
else:
    print("Failed to create cylindrical hub")


Cylindrical Core Hub created successfully!

Hub only (cylindrical core):
+


In [10]:
# Create miter-cut struts for the cylindrical hub
miter_struts_at_vertex = []

for edge_idx in connected_edges:
    v1_idx, v2_idx = edges[edge_idx]
    start = vertices[v1_idx]
    end = vertices[v2_idx]
    
    strut = create_miter_cut_strut(
        start, end, strut_width, strut_depth,
        hub_inset, hub_inset, dome_center
    )
    miter_struts_at_vertex.append(strut)

miter_struts_compound = cq.Compound.makeCompound(miter_struts_at_vertex)

print("Miter-cut struts + Cylindrical Hub:")
if cylindrical_hub is not None:
    show(miter_struts_compound, cylindrical_hub)


Miter-cut struts + Cylindrical Hub:
++


In [11]:
# Analyze miter cut angles for the struts at this vertex
print("\nMiter Cut Analysis:")
print("="*50)

radial = normalize(sub(test_vertex, dome_center))

for i, edge_idx in enumerate(connected_edges):
    v1_idx, v2_idx = edges[edge_idx]
    other_idx = v2_idx if v1_idx == test_vertex_idx else v1_idx
    other_vertex = vertices[other_idx]
    
    strut_dir = normalize(sub(other_vertex, test_vertex))
    angles = compute_miter_cut_angles(strut_dir, radial)
    
    print(f"\nStrut {i+1} (to vertex {other_idx}):")
    print(f"  Miter angle: {angles['miter_angle_deg']:.1f} deg")
    print(f"  Bevel angle: {angles['bevel_angle_deg']:.1f} deg")
    print(f"  Strut-to-radial angle: {angles['strut_to_radial_angle_deg']:.1f} deg")



Miter Cut Analysis:

Strut 1 (to vertex 57):
  Miter angle: 11.7 deg
  Bevel angle: 0.0 deg
  Strut-to-radial angle: 101.7 deg

Strut 2 (to vertex 37):
  Miter angle: 10.2 deg
  Bevel angle: 0.0 deg
  Strut-to-radial angle: 100.2 deg

Strut 3 (to vertex 41):
  Miter angle: 10.2 deg
  Bevel angle: 0.0 deg
  Strut-to-radial angle: 100.2 deg


## Section 4: Full Chapel Dome Comparison

Now let's generate the full Chapel dome (8ft radius, 3V frequency) with each hub style.


In [12]:
# Generate Chapel dome with convex hull hubs (baseline)
print("Generating Chapel dome with CONVEX HULL hubs...")
struts_hull, hubs_hull, info_hull = generate_dome_with_hubs(
    radius_cm=8.0 * 30.48,  # 8ft in cm
    frequency=3,
    strut_width=4.0 * 2.54,  # 4" in cm
    strut_depth=4.0 * 2.54,
    dome_style="honeycomb",
    hub_style="convex_hull",
    strut_style="cuboid"
)

print(f"  Struts: {info_hull['num_struts_generated']}")
print(f"  Hubs: {info_hull['num_hubs_generated']}")


Generating Chapel dome with CONVEX HULL hubs...
  Struts: 286
  Hubs: 198


In [13]:
# Visualize convex hull dome
print("Chapel Dome with Convex Hull Hubs:")
show(struts_hull, hubs_hull)


Chapel Dome with Convex Hull Hubs:
++


In [14]:
# Generate Chapel dome with tapered prism hubs
print("Generating Chapel dome with TAPERED PRISM hubs...")
struts_prism, hubs_prism, info_prism = generate_dome_with_hubs(
    radius_cm=8.0 * 30.48,
    frequency=3,
    strut_width=4.0 * 2.54,
    strut_depth=4.0 * 2.54,
    dome_style="honeycomb",
    hub_style="tapered_prism",
    strut_style="cuboid"
)

print(f"  Struts: {info_prism['num_struts_generated']}")
print(f"  Hubs: {info_prism['num_hubs_generated']}")


Generating Chapel dome with TAPERED PRISM hubs...
  Struts: 286
  Hubs: 198


In [15]:
# Visualize tapered prism dome
print("Chapel Dome with Tapered Prism Hubs:")
show(struts_prism, hubs_prism)


Chapel Dome with Tapered Prism Hubs:
c+


In [1]:
# Generate Chapel dome with cylindrical core hubs + miter-cut struts
print("Generating Chapel dome with CYLINDRICAL CORE hubs + miter-cut struts...")
struts_cyl, hubs_cyl, info_cyl = generate_dome_with_hubs(
    radius_cm=8.0 * 30.48,
    frequency=3,
    strut_width=4.0 * 2.54,
    strut_depth=4.0 * 2.54,
    dome_style="honeycomb",
    hub_style="cylindrical_core",
    strut_style="miter_cut"
)

print(f"  Struts: {info_cyl['num_struts_generated']}")
print(f"  Hubs: {info_cyl['num_hubs_generated']}")


Generating Chapel dome with CYLINDRICAL CORE hubs + miter-cut struts...


NameError: name 'generate_dome_with_hubs' is not defined

In [16]:
# Visualize cylindrical core dome
print("Chapel Dome with Cylindrical Core Hubs:")
show(struts_cyl, hubs_cyl)


Chapel Dome with Cylindrical Core Hubs:
++


## Section 5: Manufacturability Analysis - Unique Parts Count

A critical question for manufacturing is: **How many unique parts do we need to make?**

We'll analyze:
1. **Unique strut types** - grouped by length and cut angles
2. **Unique hub types** - grouped by number of struts and angles between them


In [16]:
from collections import Counter

def analyze_unique_struts(vertices, edges, dome_center, strut_style="cuboid", 
                          length_tol_cm=0.1, angle_tol_deg=0.5):
    """
    Analyze struts and group by similar dimensions.
    
    For cuboid struts: group by length only (ends are perpendicular cuts)
    For miter-cut struts: group by length + cut angles at both ends
    """
    strut_signatures = []
    
    for v1_idx, v2_idx in edges:
        v1 = vertices[v1_idx]
        v2 = vertices[v2_idx]
        
        # Strut length
        length = norm(sub(v2, v1))
        length_rounded = round(length / length_tol_cm) * length_tol_cm
        
        if strut_style == "miter_cut":
            # Compute cut angles at both ends
            radial_v1 = normalize(sub(v1, dome_center))
            radial_v2 = normalize(sub(v2, dome_center))
            
            strut_dir = normalize(sub(v2, v1))
            
            angles_v1 = compute_miter_cut_angles(strut_dir, radial_v1)
            angles_v2 = compute_miter_cut_angles(scale(strut_dir, -1), radial_v2)
            
            angle1 = round(angles_v1['miter_angle_deg'] / angle_tol_deg) * angle_tol_deg
            angle2 = round(angles_v2['miter_angle_deg'] / angle_tol_deg) * angle_tol_deg
            
            # Sort angles so (30, 45) == (45, 30) for symmetric struts
            angle_pair = tuple(sorted([angle1, angle2]))
            
            signature = (length_rounded, angle_pair)
        else:
            # Cuboid struts - just length matters
            signature = (length_rounded,)
        
        strut_signatures.append(signature)
    
    # Count unique signatures
    signature_counts = Counter(strut_signatures)
    
    return {
        'total_struts': len(edges),
        'unique_types': len(signature_counts),
        'signature_counts': dict(signature_counts),
    }


def analyze_unique_hubs(vertices, edges, vertex_to_edges, dome_center, 
                        angle_tol_deg=1.0):
    """
    Analyze hubs and group by similar configurations.
    
    Hubs are characterized by:
    - Number of struts
    - Angles between adjacent struts (sorted)
    """
    hub_signatures = []
    
    for v_idx, edge_indices in vertex_to_edges.items():
        vertex = vertices[v_idx]
        num_struts = len(edge_indices)
        
        if num_struts < 2:
            continue
        
        # Get strut directions
        directions = []
        for edge_idx in edge_indices:
            v1_idx, v2_idx = edges[edge_idx]
            other_idx = v2_idx if v1_idx == v_idx else v1_idx
            other = vertices[other_idx]
            direction = normalize(sub(other, vertex))
            directions.append(direction)
        
        # Compute angles between all pairs of struts
        angles = []
        for i in range(len(directions)):
            for j in range(i+1, len(directions)):
                d1, d2 = directions[i], directions[j]
                cos_angle = d1[0]*d2[0] + d1[1]*d2[1] + d1[2]*d2[2]
                angle_deg = math.degrees(math.acos(max(-1, min(1, cos_angle))))
                angle_rounded = round(angle_deg / angle_tol_deg) * angle_tol_deg
                angles.append(angle_rounded)
        
        # Sort angles to create a canonical signature
        angles_sorted = tuple(sorted(angles))
        
        signature = (num_struts, angles_sorted)
        hub_signatures.append(signature)
    
    signature_counts = Counter(hub_signatures)
    
    return {
        'total_hubs': len(hub_signatures),
        'unique_types': len(signature_counts),
        'signature_counts': dict(signature_counts),
    }


In [17]:
# Analyze Chapel dome parts
print("="*70)
print("MANUFACTURABILITY ANALYSIS - CHAPEL DOME (8ft, 3V)")
print("="*70)

# Get dome geometry
radius_cm_analysis = 8.0 * 30.48
vertices_analysis, faces_analysis, edges_analysis = generate_honeycomb_dome(radius_cm_analysis, 3, 0.5)
vertex_to_edges_analysis = build_vertex_to_edges_map(edges_analysis)
dome_center_analysis = (0, 0, 0)

print(f"\nDome Statistics:")
print(f"  Total vertices: {len(vertices_analysis)}")
print(f"  Total edges (struts): {len(edges_analysis)}")
print(f"  Total faces (windows): {len(faces_analysis)}")


MANUFACTURABILITY ANALYSIS - CHAPEL DOME (8ft, 3V)

Dome Statistics:
  Total vertices: 198
  Total edges (struts): 286
  Total faces (windows): 89


In [18]:
# Analyze struts for cuboid style (Option 2: Tapered Prism)
print("\n" + "-"*70)
print("OPTION 2: TAPERED PRISM HUBS + CUBOID STRUTS")
print("-"*70)

cuboid_analysis = analyze_unique_struts(vertices_analysis, edges_analysis, dome_center_analysis, "cuboid")
print(f"\nStrut Analysis:")
print(f"  Total struts: {cuboid_analysis['total_struts']}")
print(f"  Unique strut types: {cuboid_analysis['unique_types']}")
print(f"\n  Cut list (length -> count):")
for sig, count in sorted(cuboid_analysis['signature_counts'].items()):
    length_cm = sig[0]
    length_in = length_cm / 2.54
    print(f"    {length_cm:.1f} cm ({length_in:.1f} in): {count} pieces")



----------------------------------------------------------------------
OPTION 2: TAPERED PRISM HUBS + CUBOID STRUTS
----------------------------------------------------------------------

Strut Analysis:
  Total struts: 286
  Unique strut types: 6

  Cut list (length -> count):
    37.3 cm (14.7 in): 36 pieces
    41.4 cm (16.3 in): 72 pieces
    42.2 cm (16.6 in): 40 pieces
    42.6 cm (16.8 in): 68 pieces
    42.7 cm (16.8 in): 34 pieces
    44.4 cm (17.5 in): 36 pieces


In [19]:
# Analyze hubs for tapered prism
hub_analysis = analyze_unique_hubs(vertices_analysis, edges_analysis, vertex_to_edges_analysis, dome_center_analysis)
print(f"\nHub Analysis:")
print(f"  Total hubs: {hub_analysis['total_hubs']}")
print(f"  Unique hub types: {hub_analysis['unique_types']}")
print(f"\n  Hub types (struts, angles) -> count:")
for sig, count in sorted(hub_analysis['signature_counts'].items()):
    num_struts = sig[0]
    angles = sig[1]
    angles_str = ", ".join(f"{a:.0f} deg" for a in angles)
    print(f"    {num_struts}-strut hub [{angles_str}]: {count} pieces")



Hub Analysis:
  Total hubs: 198
  Unique hub types: 11

  Hub types (struts, angles) -> count:
    2-strut hub [108 deg]: 4 pieces
    2-strut hub [115 deg]: 4 pieces
    2-strut hub [117 deg]: 4 pieces
    2-strut hub [118 deg]: 2 pieces
    2-strut hub [120 deg]: 4 pieces
    2-strut hub [121 deg]: 4 pieces
    3-strut hub [108 deg, 125 deg, 125 deg]: 36 pieces
    3-strut hub [115 deg, 115 deg, 127 deg]: 32 pieces
    3-strut hub [117 deg, 120 deg, 121 deg]: 64 pieces
    3-strut hub [118 deg, 120 deg, 120 deg]: 32 pieces
    3-strut hub [119 deg, 119 deg, 119 deg]: 12 pieces


In [20]:
# Analyze struts for miter-cut style (Option 3: Cylindrical Core)
print("\n" + "-"*70)
print("OPTION 3: CYLINDRICAL CORE HUBS + MITER-CUT STRUTS")
print("-"*70)

miter_analysis = analyze_unique_struts(vertices_analysis, edges_analysis, dome_center_analysis, "miter_cut")
print(f"\nStrut Analysis:")
print(f"  Total struts: {miter_analysis['total_struts']}")
print(f"  Unique strut types: {miter_analysis['unique_types']}")
print(f"\n  Cut configurations (length, angles) -> count:")
for sig, count in sorted(miter_analysis['signature_counts'].items()):
    length_cm = sig[0]
    length_in = length_cm / 2.54
    angles = sig[1]
    print(f"    {length_cm:.1f} cm ({length_in:.1f} in) @ ({angles[0]:.1f} deg, {angles[1]:.1f} deg): {count} pieces")



----------------------------------------------------------------------
OPTION 3: CYLINDRICAL CORE HUBS + MITER-CUT STRUTS
----------------------------------------------------------------------

Strut Analysis:
  Total struts: 286
  Unique strut types: 6

  Cut configurations (length, angles) -> count:
    37.3 cm (14.7 in) @ (4.0 deg, 4.5 deg): 36 pieces
    41.4 cm (16.3 in) @ (4.0 deg, 6.0 deg): 72 pieces
    42.2 cm (16.6 in) @ (5.0 deg, 5.0 deg): 40 pieces
    42.6 cm (16.8 in) @ (5.0 deg, 5.5 deg): 68 pieces
    42.7 cm (16.8 in) @ (5.0 deg, 5.0 deg): 34 pieces
    44.4 cm (17.5 in) @ (5.0 deg, 5.5 deg): 36 pieces


In [21]:
# For cylindrical hubs, they're all the same shape! Just cylinders at different positions.
print(f"\nHub Analysis (Cylindrical):")

# Count hubs by number of struts
hub_counts_by_struts = Counter(len(v) for v in vertex_to_edges_analysis.values())
total_hubs = sum(1 for v in vertex_to_edges_analysis.values() if len(v) >= 2)

print(f"  Total hubs: {total_hubs}")
print(f"  Unique hub shapes: 2")  # 2-strut and 3-strut cylinders (same shape, just position differs)
print(f"\n  Hub distribution:")
for num_struts, count in sorted(hub_counts_by_struts.items()):
    if num_struts >= 2:
        print(f"    {num_struts}-strut cylinder hubs: {count} pieces")



Hub Analysis (Cylindrical):
  Total hubs: 198
  Unique hub shapes: 2

  Hub distribution:
    2-strut cylinder hubs: 22 pieces
    3-strut cylinder hubs: 176 pieces


In [22]:
# Summary comparison
print("\n" + "="*70)
print("SUMMARY COMPARISON")
print("="*70)

print(f"\n{'Metric':<35} {'Tapered Prism':>15} {'Cylindrical Core':>18}")
print("-"*70)
print(f"{'Total struts':<35} {cuboid_analysis['total_struts']:>15} {miter_analysis['total_struts']:>18}")
print(f"{'Unique strut types':<35} {cuboid_analysis['unique_types']:>15} {miter_analysis['unique_types']:>18}")
print(f"{'Total hubs':<35} {hub_analysis['total_hubs']:>15} {total_hubs:>18}")
print(f"{'Unique hub types':<35} {hub_analysis['unique_types']:>15} {2:>18}")
print("-"*70)
total_unique_prism = cuboid_analysis['unique_types'] + hub_analysis['unique_types']
total_unique_cyl = miter_analysis['unique_types'] + 2
print(f"{'TOTAL UNIQUE PARTS':<35} {total_unique_prism:>15} {total_unique_cyl:>18}")



SUMMARY COMPARISON

Metric                                Tapered Prism   Cylindrical Core
----------------------------------------------------------------------
Total struts                                    286                286
Unique strut types                                6                  6
Total hubs                                      198                198
Unique hub types                                 11                  2
----------------------------------------------------------------------
TOTAL UNIQUE PARTS                               17                  8


In [23]:
# Manufacturing notes
print("\n" + "="*70)
print("MANUFACTURING NOTES")
print("="*70)
print("""
TAPERED PRISM (Option 2):
  + Simpler strut cuts (perpendicular only)
  + Clean flat inner surface for aesthetics
  - Many unique hub shapes (one for each unique angle configuration)
  - Hub shapes are irregular polyhedra (harder to manufacture)
  - Best for: 3D printing or CNC machining hubs

CYLINDRICAL CORE (Option 3):
  + Only 2 unique hub shapes (cylinders are easy to manufacture)
  + Hubs can be turned on a lathe or 3D printed easily
  - More unique strut types (length + two angle combinations)
  - Compound miter cuts require CNC or careful jig setup
  - Best for: Production runs where you can batch strut cuts
""")



MANUFACTURING NOTES

TAPERED PRISM (Option 2):
  + Simpler strut cuts (perpendicular only)
  + Clean flat inner surface for aesthetics
  - Many unique hub shapes (one for each unique angle configuration)
  - Hub shapes are irregular polyhedra (harder to manufacture)
  - Best for: 3D printing or CNC machining hubs

CYLINDRICAL CORE (Option 3):
  + Only 2 unique hub shapes (cylinders are easy to manufacture)
  + Hubs can be turned on a lathe or 3D printed easily
  - More unique strut types (length + two angle combinations)
  - Compound miter cuts require CNC or careful jig setup
  - Best for: Production runs where you can batch strut cuts



## Section 6: Detailed Cut Lists for Manufacturing

Generate detailed cut lists that can be used for actual manufacturing.


In [24]:
# Generate detailed cut list for cuboid struts
print("="*70)
print("DETAILED CUT LIST - CUBOID STRUTS (Option 2: Tapered Prism)")
print("="*70)
print(f"\nStrut cross-section: 4 in x 4 in (10.16 cm x 10.16 cm)")
print(f"Cut type: Perpendicular (90 deg) at both ends")
print(f"\n{'Type':<6} {'Length (cm)':<12} {'Length (in)':<12} {'Count':<8} {'Notes'}")
print("-"*70)

for i, (sig, count) in enumerate(sorted(cuboid_analysis['signature_counts'].items()), 1):
    length_cm = sig[0]
    length_in = length_cm / 2.54
    print(f"A{i:<5} {length_cm:<12.1f} {length_in:<12.1f} {count:<8} Square cut both ends")

total_length_cm = sum(sig[0] * count for sig, count in cuboid_analysis['signature_counts'].items())
total_length_ft = total_length_cm / 30.48
print("-"*70)
print(f"TOTAL: {cuboid_analysis['total_struts']} struts, {total_length_ft:.0f} linear feet of material")


DETAILED CUT LIST - CUBOID STRUTS (Option 2: Tapered Prism)

Strut cross-section: 4 in x 4 in (10.16 cm x 10.16 cm)
Cut type: Perpendicular (90 deg) at both ends

Type   Length (cm)  Length (in)  Count    Notes
----------------------------------------------------------------------
A1     37.3         14.7         36       Square cut both ends
A2     41.4         16.3         72       Square cut both ends
A3     42.2         16.6         40       Square cut both ends
A4     42.6         16.8         68       Square cut both ends
A5     42.7         16.8         34       Square cut both ends
A6     44.4         17.5         36       Square cut both ends
----------------------------------------------------------------------
TOTAL: 286 struts, 392 linear feet of material


In [25]:
# Generate detailed cut list for miter-cut struts
print("\n" + "="*70)
print("DETAILED CUT LIST - MITER-CUT STRUTS (Option 3: Cylindrical Core)")
print("="*70)
print(f"\nStrut cross-section: 4 in x 4 in (10.16 cm x 10.16 cm)")
print(f"Cut type: Compound miter (angles vary by strut type)")
print(f"\n{'Type':<6} {'Length':<12} {'Angle A':<12} {'Angle B':<12} {'Count':<8}")
print("-"*70)

for i, (sig, count) in enumerate(sorted(miter_analysis['signature_counts'].items()), 1):
    length_cm = sig[0]
    length_in = length_cm / 2.54
    angles = sig[1]
    print(f"M{i:<5} {length_in:<12.1f} in {angles[0]:<12.1f} deg {angles[1]:<12.1f} deg {count:<8}")

print("-"*70)
print(f"TOTAL: {miter_analysis['total_struts']} struts")



DETAILED CUT LIST - MITER-CUT STRUTS (Option 3: Cylindrical Core)

Strut cross-section: 4 in x 4 in (10.16 cm x 10.16 cm)
Cut type: Compound miter (angles vary by strut type)

Type   Length       Angle A      Angle B      Count   
----------------------------------------------------------------------
M1     14.7         in 4.0          deg 4.5          deg 36      
M2     16.3         in 4.0          deg 6.0          deg 72      
M3     16.6         in 5.0          deg 5.0          deg 40      
M4     16.8         in 5.0          deg 5.5          deg 68      
M5     16.8         in 5.0          deg 5.0          deg 34      
M6     17.5         in 5.0          deg 5.5          deg 36      
----------------------------------------------------------------------
TOTAL: 286 struts


In [26]:
# Generate hub list for tapered prism hubs
print("\n" + "="*70)
print("DETAILED HUB LIST - TAPERED PRISM HUBS (Option 2)")
print("="*70)
print(f"\nHub material: Custom 3D printed or CNC machined")
print(f"\n{'Type':<6} {'Struts':<8} {'Angles':<40} {'Count':<8}")
print("-"*70)

for i, (sig, count) in enumerate(sorted(hub_analysis['signature_counts'].items()), 1):
    num_struts = sig[0]
    angles = sig[1]
    angles_str = ", ".join(f"{a:.0f} deg" for a in angles)
    print(f"H{i:<5} {num_struts:<8} {angles_str:<40} {count:<8}")

print("-"*70)
print(f"TOTAL: {hub_analysis['total_hubs']} hubs in {hub_analysis['unique_types']} unique types")



DETAILED HUB LIST - TAPERED PRISM HUBS (Option 2)

Hub material: Custom 3D printed or CNC machined

Type   Struts   Angles                                   Count   
----------------------------------------------------------------------
H1     2        108 deg                                  4       
H2     2        115 deg                                  4       
H3     2        117 deg                                  4       
H4     2        118 deg                                  2       
H5     2        120 deg                                  4       
H6     2        121 deg                                  4       
H7     3        108 deg, 125 deg, 125 deg                36      
H8     3        115 deg, 115 deg, 127 deg                32      
H9     3        117 deg, 120 deg, 121 deg                64      
H10    3        118 deg, 120 deg, 120 deg                32      
H11    3        119 deg, 119 deg, 119 deg                12      
------------------------------------

In [27]:
# Generate hub list for cylindrical hubs
print("\n" + "="*70)
print("DETAILED HUB LIST - CYLINDRICAL CORE HUBS (Option 3)")
print("="*70)
print(f"\nHub material: Turned on lathe, or 3D printed")
print(f"Cylinder diameter: ~{4.0 * 0.8:.1f} in (based on hub_inset * 0.8)")
print(f"Cylinder height: 4 in (strut depth)")
print(f"\n{'Type':<10} {'Description':<35} {'Count':<8}")
print("-"*70)

count_2strut = hub_counts_by_struts.get(2, 0)
count_3strut = hub_counts_by_struts.get(3, 0)

if count_2strut > 0:
    print(f"{'CYL-2':<10} {'2-strut cylinder (base edge)':<35} {count_2strut:<8}")
if count_3strut > 0:
    print(f"{'CYL-3':<10} {'3-strut cylinder (interior)':<35} {count_3strut:<8}")

print("-"*70)
print(f"TOTAL: {total_hubs} hubs in 2 unique shapes")



DETAILED HUB LIST - CYLINDRICAL CORE HUBS (Option 3)

Hub material: Turned on lathe, or 3D printed
Cylinder diameter: ~3.2 in (based on hub_inset * 0.8)
Cylinder height: 4 in (strut depth)

Type       Description                         Count   
----------------------------------------------------------------------
CYL-2      2-strut cylinder (base edge)        22      
CYL-3      3-strut cylinder (interior)         176     
----------------------------------------------------------------------
TOTAL: 198 hubs in 2 unique shapes


## Conclusion

Both hub designs have trade-offs:

| Aspect | Tapered Prism | Cylindrical Core |
|--------|---------------|------------------|
| **Unique strut types** | Fewer (length only) | More (length + angles) |
| **Unique hub types** | Many (complex shapes) | Just 2 (simple cylinders) |
| **Strut manufacturing** | Simple perpendicular cuts | Compound miter cuts (CNC) |
| **Hub manufacturing** | 3D printing or CNC | Lathe or 3D printing |
| **Interior aesthetics** | Clean flat triangular faces | Cylindrical protrusions |

**Recommendation:**
- For **small-scale production** or one-off builds: **Tapered Prism** is likely easier since strut cuts are simple
- For **larger production runs**: **Cylindrical Core** may be better since you only need 2 hub molds/designs
